In [ ]:
%matplotlib inline


变分自动编码器
=======================
**翻译者**: `Antares博士 <http://www.studyai.com/antares>`_

这是Kingma和Welling对论文(`[随机梯度vb和变分自动编码器] <http://arxiv.org/abs/1312.6114>`__)的改进实现。
它使用ReLUs激活函数和Adam优化器，而不是sigmoids激活函数和Adagrad优化器。这些改动使网络收敛得更快。



导入依赖包
-----------------------



In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid

全局参数
-----------------------



In [ ]:
parser = argparse.ArgumentParser(description='VAE MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='用于训练的输入批次大小 (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='用于训练的回合数 (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='启用 CUDA 训练')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='随机数种子 (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='每隔多少个批次记录一次训练状态')
args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

准备数据集和数据加载器
-----------------------



In [ ]:
# 指定计算设备
torch.manual_seed(12321236)
device = torch.device("cuda" if args.cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/mnist', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/mnist', train=False, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

显示图像数据集
-----------------------



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# 打开交互模式
plt.ion()
# 忽略 warnings
import warnings
warnings.filterwarnings("ignore")

# 用于显示一张图像的函数
def imshow(img,title=None):
    npimg = img.numpy()
    plt.figure(figsize=[6.5,2.5])
    plt.tight_layout()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.title(title)
    plt.show()
    plt.pause(0.1)

# 获取一个批次的图像，一次迭代取出batch_size张图片
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images[:16].size())
# 显示16张图像
imshow(make_grid(images[:16]), "The Original Digits Images")

定义变分自动编码器模型
-----------------------



In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

model = VAE().to(device)

# 打印输出网络结构
print(model)

定义损失函数和优化器
-----------------------



In [ ]:
# 损失函数: Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

# 优化器
optimizer = optim.Adam(model.parameters(), lr=1e-3)

定义训练过程
-----------------------



In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

定义测试过程
-----------------------



In [ ]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
                # 下面这个函数保存图像的时候有问题，不知啥原因
                # save_image(comparison.cpu(), 'results/reconstruction_' + str(epoch) + '.png', nrow=n)
                if epoch % args.epochs == 0:
                    # 显示最后的一次epoch后解码图像与原图像作对比
                    imshow(make_grid(comparison.cpu()), "First Row:Original Images,Sencond Row:Decoded Images")

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

开始训练和测试
-----------------------



In [ ]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)

从高斯分布的随机数据解码
---------------------------------



In [ ]:
with torch.no_grad():
    sample = torch.randn(32, 20).to(device)
    sample = model.decode(sample).cpu()
    # 下面这个函数保存图像的时候有问题，不知啥原因
    # save_image(sample.view(32, 1, 28, 28), 'results/sample_' + '.png')
    if epoch % args.epochs == 0:
        imshow(make_grid(sample.view(32, 1, 28, 28).cpu()), "Decoded Images From Gaussian Data")